<a href="https://colab.research.google.com/github/JAdrison/evolution-liftaads/blob/main/Agente_In_Python_Google_%2B_Alura.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aula 01 - LLM in python > Gemini + Alura


### Download dos app

In [ ]:
!pip install -q --upgrade langchain langchain-google-genai google-generativeai

### Importação da API Key

In [11]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI

Google_Api_Key = userdata.get('GOOGLE_API_KEY')

### Conexão com o Gemini

In [28]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.0,
    api_key=Google_Api_Key
)

### Pergunta a LLM + Resposta





In [29]:
resposta_01_test = llm.invoke("Qual a melho comida do Brasil?")

print (resposta_01_test)

content='Essa é uma pergunta difícil, pois **não existe uma única "melhor" comida do Brasil**! A culinária brasileira é incrivelmente rica e diversa, variando muito de região para região, e o que é "melhor" depende totalmente do gosto pessoal.\n\nNo entanto, posso listar algumas das comidas mais **icônicas, amadas e representativas** do Brasil, que são frequentemente citadas como favoritas:\n\n1.  **Feijoada:** Considerada por muitos o prato nacional. É um cozido robusto de feijão preto com diversas carnes de porco (linguiça, costela, carne seca, etc.), servido com arroz branco, farofa, couve refogada e laranja. É um prato de festa, geralmente consumido aos sábados.\n\n2.  **Churrasco:** Especialmente forte no Sul do país, mas amado em todo o Brasil. Carnes de alta qualidade assadas na brasa, muitas vezes servidas em espetos, acompanhadas de arroz, farofa, vinagrete e salada.\n\n3.  **Moqueca (Capixaba ou Baiana):** Um ensopado de peixe e frutos do mar.\n    *   **Moqueca Capixaba:** M

### Prompt do Agente

In [32]:
TRIAGEM_PROMPT = (
    "Você é um triador de Service Desk para políticas internas da empresa Carraro Desenvolvimento. "
    "Dada a mensagem do usuário, retorne SOMENTE um JSON com:\n"
    "{\n"
    '  "decisao": "AUTO_RESOLVER" | "PEDIR_INFO" | "ABRIR_CHAMADO",\n'
    '  "urgencia": "BAIXA" | "MEDIA" | "ALTA",\n'
    '  "campos_faltantes": ["..."]\n'
    "}\n"
    "Regras:\n"
    '- **AUTO_RESOLVER**: Perguntas claras sobre regras ou procedimentos descritos nas políticas (Ex: "Posso reembolsar a internet do meu home office?", "Como funciona a política de alimentação em viagens?").\n'
    '- **PEDIR_INFO**: Mensagens vagas ou que faltam informações para identificar o tema ou contexto (Ex: "Preciso de ajuda com uma política", "Tenho uma dúvida geral").\n'
    '- **ABRIR_CHAMADO**: Pedidos de exceção, liberação, aprovação ou acesso especial, ou quando o usuário explicitamente pede para abrir um chamado (Ex: "Quero exceção para trabalhar 5 dias remoto.", "Solicito liberação para anexos externos.", "Por favor, abra um chamado para o RH.").'
    "Analise a mensagem e decida a ação mais apropriada."
)

In [36]:
from pydantic import BaseModel, Field
from typing import Literal, List, Dict

class TriagemOut(BaseModel):
    decisao: Literal["AUTO_RESOLVER", "PEDIR_INFO", "ABRIR_CHAMADO"]
    urgencia: Literal["BAIXA", "MEDIA", "ALTA"]
    campos_faltantes: List[str] = Field(default_factory=list)

### LLM ESPECIALISTA EM TRIAGEM (NOVA LLM )

In [38]:
llm_triagem = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.0,
    api_key=Google_Api_Key
)

In [39]:
from langchain_core.messages import SystemMessage, HumanMessage

triagem_chain = llm_triagem.with_structured_output(TriagemOut)

def triagem(mensagem: str) -> Dict:
    saida: TriagemOut = triagem_chain.invoke([
        SystemMessage(content=TRIAGEM_PROMPT),
        HumanMessage(content=mensagem)
    ])

    return saida.model_dump()

### Perguntas

In [40]:
testes = ["Posso reembolsar a internet?",
          "Quero mais 5 dias de trabalho remoto. Como faço?",
          "Posso reembolsar cursos ou treinamentos da Alura?",
          "Quantas capivaras tem no Rio Pinheiros?"]

### Resposta

In [41]:
for msg_teste in testes:
    print(f"Pergunta: {msg_teste}\n -> Resposta: {triagem(msg_teste)}\n")

Pergunta: Posso reembolsar a internet?
 -> Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

Pergunta: Quero mais 5 dias de trabalho remoto. Como faço?
 -> Resposta: {'decisao': 'ABRIR_CHAMADO', 'urgencia': 'MEDIA', 'campos_faltantes': []}

Pergunta: Posso reembolsar cursos ou treinamentos da Alura?
 -> Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

Pergunta: Quantas capivaras tem no Rio Pinheiros?
 -> Resposta: {'decisao': 'PEDIR_INFO', 'urgencia': 'BAIXA', 'campos_faltantes': ['informação sobre a política interna']}

